In [1]:
from pyspark.context import SparkContext, SparkConf
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F

conf=SparkConf()

conf.set('spark.sql.join.preferSortMergeJoin', 'true')
conf.set('spark.sql.autoBroadcastJoinThreshold', 0)
conf.set('spark.driver.memory', '8g')
conf.set("spark.sql.parquet.binaryAsString","true")

sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)


In [18]:
flights = spark.read.parquet("flights_paritioned_year_month/").filter("origin = 'LAX'")
# flights.write.partitionBy("year", "month").parquet("flights_paritioned_year_month/")

In [20]:
airports = spark.read.csv("flights/airports.csv", header=True)

+----+--------------------+----------------+-----+-------+-----------+------------+
|iata|             airport|            city|state|country|        lat|        long|
+----+--------------------+----------------+-----+-------+-----------+------------+
| 00M|            Thigpen |     Bay Springs|   MS|    USA|31.95376472|-89.23450472|
| 00R|Livingston Municipal|      Livingston|   TX|    USA|30.68586111|-95.01792778|
| 00V|         Meadow Lake|Colorado Springs|   CO|    USA|38.94574889|-104.5698933|
| 01G|        Perry-Warsaw|           Perry|   NY|    USA|42.74134667|-78.05208056|
+----+--------------------+----------------+-----+-------+-----------+------------+
only showing top 4 rows



In [23]:
flights.registerTempTable("flights")
airports.registerTempTable("airports")
sql = """
SELECT flights.carrier, airports.iata, airports.airport as destination
FROM flights 
LEFT JOIN airports
on flights.dest = airports.iata

"""
spark.sql(sql).write.parquet("deletelax")

+-------+----+--------------------+
|carrier|iata|         destination|
+-------+----+--------------------+
|     WN| ABQ|Albuquerque Inter...|
|     WN| ABQ|Albuquerque Inter...|
|     WN| ABQ|Albuquerque Inter...|
|     WN| ABQ|Albuquerque Inter...|
|     WN| AUS|Austin-Bergstrom ...|
|     WN| BNA|Nashville Interna...|
|     WN| BNA|Nashville Interna...|
|     WN| BNA|Nashville Interna...|
|     WN| BWI|Baltimore-Washing...|
|     WN| BWI|Baltimore-Washing...|
+-------+----+--------------------+

